# Section 1-1 - Basic Neural Network - Titanic

In the previous section, we simply iterated through randomly-generated matrices and chose the best-performing one. We build on this approach by reducing loss in a systematic way via stochastic gradient descent. In particular, we'll be using TensorFlow, an open source library developed by Google, and Keras, a high-level wrapper on top of TensorFlow.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('data/titanic.csv')

In [2]:
df_train = df.iloc[:712, :]

df_train = df_train.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

age_mean = df_train['Age'].mean()
df_train['Age'] = df_train['Age'].fillna(age_mean)
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1}).astype(int)

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
scaler = StandardScaler()

X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [3]:
df_test = df.iloc[712:, :]

df_test = df_test.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)

X_test = scaler.transform(df_test[features].values)
y_test = df_test['Survived'].values

## Benchmark

In [4]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['Survived'].values)

y_prediction = model.predict(X_test)
print "\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.832402234637


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


## 1-layer Neural Network

Instead of generating a linear stack of layers with Numpy, we'll be implementing our model using Keras. We initialize our model, add a layer that inputs vectors of length 6 and outputs vectors of length 2, and finally add a softmax layer. We configure the learning process in the compilation step by specifying the optimizer, loss function and performance metric.

Stochastic gradient descent acts by changing the weights gradually in the 'direction' that decreases the average loss. In other words, a particular weight would be increased if acts to decrease loss, or the weight decreased if it acts to increase loss. TensorFlow does the heavy-lifting by efficiently handling these numerical computations under the hood. A simple example of gradient descent is illustrated in the Appendix.

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(input_dim=6, output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print '\ntime taken %s seconds' % str(time() - start)

Using TensorFlow backend.


Epoch 1/10
712/712 [==============================] - 0s - loss: 0.6828 - acc: 0.7191     
Epoch 2/10
712/712 [==============================] - 0s - loss: 0.6620 - acc: 0.7233     
Epoch 3/10
712/712 [==============================] - 0s - loss: 0.6441 - acc: 0.7317     
Epoch 4/10
712/712 [==============================] - 0s - loss: 0.6266 - acc: 0.7360     
Epoch 5/10
712/712 [==============================] - 0s - loss: 0.6108 - acc: 0.7444     
Epoch 6/10
712/712 [==============================] - 0s - loss: 0.5967 - acc: 0.7472     
Epoch 7/10
712/712 [==============================] - 0s - loss: 0.5840 - acc: 0.7584     
Epoch 8/10
712/712 [==============================] - 0s - loss: 0.5732 - acc: 0.7626     
Epoch 9/10
712/712 [==============================] - 0s - loss: 0.5623 - acc: 0.7640     
Epoch 10/10
712/712 [==============================] - 0s - loss: 0.5530 - acc: 0.7612     

time taken 0.753010988235 seconds


In [6]:
y_prediction = model.predict_classes(X_test)
print "\n\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

 32/179 [====>.........................] - ETA: 0s

accuracy 0.826815642458


We notice that the loss reduces systematically as the model 'learns' from the data. The rate of loss reduction, however, seems to indicate that loss could be further reduced.

## 2-layer Neural Network

In [7]:
start = time()

model = Sequential()
model.add(Dense(input_dim=6, output_dim=100))
model.add(Dense(output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print '\ntime taken %s seconds' % str(time() - start)

Epoch 1/10
712/712 [==============================] - 0s - loss: 0.5915 - acc: 0.7514     
Epoch 2/10
712/712 [==============================] - 0s - loss: 0.5325 - acc: 0.7626     
Epoch 3/10
712/712 [==============================] - 0s - loss: 0.5006 - acc: 0.7851     
Epoch 4/10
712/712 [==============================] - 0s - loss: 0.4844 - acc: 0.7893     
Epoch 5/10
712/712 [==============================] - 0s - loss: 0.4747 - acc: 0.7823     
Epoch 6/10
712/712 [==============================] - 0s - loss: 0.4691 - acc: 0.7865     
Epoch 7/10
712/712 [==============================] - 0s - loss: 0.4655 - acc: 0.7865     
Epoch 8/10
712/712 [==============================] - 0s - loss: 0.4627 - acc: 0.7879     
Epoch 9/10
712/712 [==============================] - 0s - loss: 0.4606 - acc: 0.7851     
Epoch 10/10
712/712 [==============================] - 0s - loss: 0.4595 - acc: 0.7893     

time taken 0.86990904808 seconds


In [8]:
y_prediction = model.predict_classes(X_test)
print "\n\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

 32/179 [====>.........................] - ETA: 0s

accuracy 0.826815642458


The loss reduction 'flattens out' more compared to the 1-layer example. The accuracy, however, remains at 82.7%.

## 3-layer Neural Network

In [9]:
start = time()

model = Sequential()
model.add(Dense(input_dim=6, output_dim=100))
model.add(Dense(output_dim=100))
model.add(Dense(output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot)

print '\ntime taken %s seconds' % str(time() - start)

Epoch 1/10
712/712 [==============================] - 0s - loss: 0.5742 - acc: 0.7346     
Epoch 2/10
712/712 [==============================] - 0s - loss: 0.5061 - acc: 0.7669     
Epoch 3/10
712/712 [==============================] - 0s - loss: 0.4809 - acc: 0.7781     
Epoch 4/10
712/712 [==============================] - 0s - loss: 0.4696 - acc: 0.7907     
Epoch 5/10
712/712 [==============================] - 0s - loss: 0.4646 - acc: 0.7865     
Epoch 6/10
712/712 [==============================] - 0s - loss: 0.4618 - acc: 0.7865     
Epoch 7/10
712/712 [==============================] - 0s - loss: 0.4599 - acc: 0.7893     
Epoch 8/10
712/712 [==============================] - 0s - loss: 0.4591 - acc: 0.7879     
Epoch 9/10
712/712 [==============================] - 0s - loss: 0.4588 - acc: 0.7865     
Epoch 10/10
712/712 [==============================] - 0s - loss: 0.4585 - acc: 0.7893     

time taken 1.18338799477 seconds


In [10]:
y_prediction = model.predict_classes(X_test)
print "\n\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

 32/179 [====>.........................] - ETA: 0s

accuracy 0.826815642458


While we're able to reduce loss on the training set a little further, the accuracy in all three cases is 82.7%. This is due to the fact that the dataset is small, and hence not much for the model to 'learn' from (or for that matter, predict on). We'll apply techniques developed so far on a much larger dataset in the next section.